<a href="https://colab.research.google.com/github/RohanGforwork/ML_text_summarization/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Necessary Modules!!!
!pip install googletrans==4.0.0-rc1 better_profanity nltk
!pip install wordfilter



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=367fefe692d5bace7e80b2819570e64feef777780c223c7d44c19d2a11dcf7e6
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperfram

ERROR: Operation cancelled by user
^C


Language translation


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from googletrans import Translator
from scipy.special import softmax

# Load model and tokenizer globally to avoid reloading on each call
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment").to(device)
model.eval()
translator = Translator()

LABELS = ["Negative", "Neutral", "Positive"]

def detect_language(text):
    """
    Detects the language of the input text.
    :param text: Input text.
    :return: Detected language and confidence score.
    """
    detected = translator.detect(text)
    return detected.lang if detected else "unknown", round(detected.confidence, 2) if detected and detected.confidence else 1.0

def translate_text(text, target_lang="en"):
    """
    Translates the text to English if it's not already in English.
    :param text: Input text.
    :param target_lang: Target language (default: English).
    :return: Translated text.
    """
    detected_lang, _ = detect_language(text)
    return translator.translate(text, dest=target_lang).text if detected_lang != target_lang else text

def analyze_sentiment(text):
    """
    Analyzes sentiment of the translated text.
    :param text: English text for sentiment analysis.
    :return: Sentiment label, confidence, and topic-based inference.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = softmax(outputs.logits.cpu().numpy()[0])

    sentiment = LABELS[scores.argmax()]
    confidence = round(scores.max(), 2)

    # Extract topic from the text
    topic = text.split('.')[0] if '.' in text else text[:50]

    # Generate inference with topic reference
    if sentiment == "Positive":
        inference = f"The discussion on '{topic}' is optimistic, suggesting support and approval."
    elif sentiment == "Negative":
        inference = f"The discussion on '{topic}' is critical, indicating concerns or disapproval."
    else:
        inference = f"The sentiment around '{topic}' is mixed, with both positive and negative perspectives present."

    return {
        "sentiment": sentiment,
        "confidence": confidence,
        "inference": inference,
        "scores": {LABELS[i]: round(float(scores[i]), 2) for i in range(len(scores))}
    }

def process_text(text):
    """
    Full pipeline: Detect language, translate (if needed), and analyze sentiment.
    :param text: User input text.
    :return: Structured output containing all analysis results.
    """
    detected_lang, lang_confidence = detect_language(text)
    translated_text = translate_text(text)
    sentiment_result = analyze_sentiment(translated_text)

    return {
        "Detected Language": detected_lang,
        "Language Confidence": lang_confidence,
        "Translated Text": translated_text,
        "Sentiment": sentiment_result["sentiment"],
        "Sentiment Confidence": sentiment_result["confidence"],
        "Inference": sentiment_result["inference"]
    }

# Example Usage
if __name__ == "__main__":
    sample_text = input("Enter text: ")
    result = process_text(sample_text)

    print(f"Detected Language: {result['Detected Language']} (Confidence: {result['Language Confidence']})")
    print(f"Translated Text: {result['Translated Text']}")
    print(f"Sentiment: {result['Sentiment']} (Confidence: {result['Sentiment Confidence']})")
    print(f"Inference: {result['Inference']}")


Enter text: ಆಡಳಿತ ಪಕ್ಷದ ಇತ್ತೀಚಿನ ನೀತಿಯಲ್ಲಿ ಬದಲಾವಣೆಗಳು ಚರ್ಚೆಗೆ ಕಾರಣವಾಗಿವೆ. ಬೆಂಬಲಿಗರು ಇದರಿಂದ ಆರ್ಥಿಕ ವೃದ್ಧಿ ಹೆಚ್ಚುತ್ತದೆ ಎಂದು ಹೇಳುತ್ತಾರೆ, ಆದರೆ ಟೀಕಾಕಾರರು ಅಸಮಾನತೆ ಹೆಚ್ಚುತ್ತದೆ ಎಂದು ವಾದಿಸುತ್ತಾರೆ. ಪ್ರಮುಖ ನಗರಗಳಲ್ಲಿ ಪ್ರತಿಭಟನೆಗಳು ಯೋಜಿತವಾಗಿವೆ."
Detected Language: kn (Confidence: 1.0)
Translated Text: Changes in the recent policy of the ruling party have led to debate.Supporters say that this will increase economic growth, but critics argue that inequality will increase.Protests are planned in major cities. ”
Sentiment: Neutral (Confidence: 0.5899999737739563)
Inference: The sentiment around 'Changes in the recent policy of the ruling party have led to debate' is mixed, with both positive and negative perspectives present.


Inference drawer!!!!!
